In [10]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



In [11]:
pip install fastparquet


Note: you may need to restart the kernel to use updated packages.


In [12]:
# In case kernel has crashed again, load the data here so you can skip all previous cells
final_data = pd.read_parquet('data/final_data_cleaned_scaled.parquet')
final_data.to_csv()

# Again since kernel sometimes crashes
all_columns = final_data.columns.tolist()
not_hot_columns = ['passenger_count', 'trip_distance', 'trip_duration', 'pickup_hour', 'pickup_day', 'peak_times', 'weekend', 'total_amount_no_tip']

final_data.head()

,passenger_count,trip_distance,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,...,Trip_Staten Island->Brooklyn,Trip_Staten Island->Manhattan,Trip_Staten Island->Queens,Trip_Staten Island->Staten Island,trip_duration,pickup_hour,pickup_day,peak_times,weekend,total_amount_no_tip
0,0.25,0.469005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.215937,0.0,1.000000,0.0,1.0,0.423341
1,0.25,0.366170,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.379359,1.0,0.833333,0.0,1.0,0.378032
2,1.00,0.964440,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.403690,0.0,1.000000,0.0,1.0,0.747735
3,0.75,0.112446,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.170316,0.0,1.000000,0.0,1.0,0.179405
4,0.25,0.242672,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.257908,0.0,1.000000,0.0,1.0,0.281922


In [13]:
final_data.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID_1.0', 'RatecodeID_2.0',
       'RatecodeID_3.0', 'RatecodeID_4.0', 'RatecodeID_5.0', 'RatecodeID_6.0',
       'RatecodeID_99.0', 'payment_type_1', 'payment_type_2', 'payment_type_3',
       'payment_type_4', 'Trip_Bronx->Bronx', 'Trip_Bronx->Brooklyn',
       'Trip_Bronx->EWR', 'Trip_Bronx->Manhattan', 'Trip_Bronx->Queens',
       'Trip_Bronx->Staten Island', 'Trip_Brooklyn->Bronx',
       'Trip_Brooklyn->Brooklyn', 'Trip_Brooklyn->EWR',
       'Trip_Brooklyn->Manhattan', 'Trip_Brooklyn->Queens',
       'Trip_Brooklyn->Staten Island', 'Trip_EWR->Brooklyn', 'Trip_EWR->EWR',
       'Trip_EWR->Manhattan', 'Trip_Manhattan->Bronx',
       'Trip_Manhattan->Brooklyn', 'Trip_Manhattan->EWR',
       'Trip_Manhattan->Manhattan', 'Trip_Manhattan->Queens',
       'Trip_Manhattan->Staten Island', 'Trip_Queens->Bronx',
       'Trip_Queens->Brooklyn', 'Trip_Queens->EWR', 'Trip_Queens->Manhattan',
       'Trip_Queens->Queens', 'Trip_Queens->Staten

In [25]:
# Creating train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [26]:
print(X_train.isnull().sum().sum())  # Telt alle missende waarden in X_train
print(X_test.isnull().sum().sum())   # Telt alle missende waarden in X_test
#dit is wel vrij hoog...

953148
946204


In [27]:
X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]  # Houd dezelfde index
X_test = X_test.dropna()
y_test = y_test.loc[X_test.index]


In [28]:
# Lastly we set create two new dataframes for the features and target variable
X = final_data.drop(columns=['total_amount_no_tip'])  # Features
y = final_data['total_amount_no_tip']  # Doelvariabele


In [29]:
print(X_train.isnull().sum().sum())  # Telt alle missende waarden in X_train
print(X_test.isnull().sum().sum())   # Telt alle missende waarden in X_test
#dit is wel vrij hoog...


0
0


In [30]:
from sklearn.metrics import mean_squared_error, r2_score
# Model trainen
model = LinearRegression()
model.fit(X_train, y_train)

# Voorspellingen maken
y_pred = model.predict(X_test)

# Model evaluatie
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.0018217521631440586
R-squared: 0.9249082870504857


In [31]:
#from sklearn.impute import SimpleImputer
#we zouden ook ipv de NaN's droppen ze vervangen door het gemiddelde
#imputer = SimpleImputer(strategy="mean")  # Kan ook "median" of "most_frequent" zijn
#X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
#X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


In [32]:
# Controleer de grootte van de trainings- en testdata
print(f"Aantal voorbeelden in de trainingsset: {X_train.shape[0]}")
print(f"Aantal voorbeelden in de testset: {X_test.shape[0]}")


Aantal voorbeelden in de trainingsset: 1521634
Aantal voorbeelden in de testset: 1521895
